## Import Libraries:

In [63]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
import numpy as np
import pandas as pd
from scipy.stats import mode
import xgboost as xgb
from xgboost import XGBRegressor,XGBClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold,StratifiedKFold
from matplotlib import  pyplot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

In [64]:
fulldata =  pd.read_csv("./basicOk.csv")
# print(fulldata.shape)
fulldata.head()

,REPORT_ID,AGENT_0,AGENT_1,AGENT_2,AGENT_3,AGENT_4,AGENT_5,AGENT_6,AGENT_7,AGENT_8,...,SALARY,day,work_latitude,work_longitude,birth_latitude,birth_longitude,is_diff1_0,is_diff1_1,is_diff1_0.1,is_diff1_1.1
0,8787,0,1,0,0,0,0,0,0,0,...,0.333333,0.466667,0.894340,0.145498,0.551743,0.433991,0,1,1,0
1,9410,0,0,0,0,0,0,0,0,0,...,0.833333,0.400000,0.868396,0.230695,0.531979,0.567527,0,1,0,1
2,22999,0,1,0,0,0,0,0,0,0,...,0.500000,0.866667,0.900189,0.120483,0.647269,0.281032,0,1,0,1
3,24820,0,1,0,0,0,0,0,0,0,...,0.166667,0.766667,0.956415,0.311420,0.791381,0.783308,0,1,1,0
4,25708,1,0,0,0,0,0,0,0,0,...,0.166667,0.500000,0.900189,0.120483,0.558606,0.335660,1,0,0,1


# 加上其他表

In [17]:
# cd_ln =  pd.read_csv("./content_ext_crd_cd_lnok.csv")
# cd_ln2 = pd.read_csv("../add/content_ext_crd_cd_lndok.csv")
cd_ln = pd.read_csv("../add/contest_ext_crd_is_creditcueok.csv")
print("cd_ln",cd_ln.shape)

ll = pd.read_csv("../rank_discre_num/contest_ext_crd_is_creditcue_discre_ok.csv")
print("ll",ll.shape)

kk = pd.read_csv("../rank_discre_num/contest_ext_crd_is_creditcue_rank_ok.csv")
print("kk",kk.shape)

# cd_ln4 = pd.read_csv("../add/contest_ext_crd_is_sharedebtok.csv")
# cd_ln5 = pd.read_csv("../add/contest_ext_crd_is_ovdsummaryok.csv")
# cd_ln6 = pd.read_csv("../add/contest_ext_crd_cd_ln_splok.csv")
# cd_ln7 = pd.read_csv("../add/contest_ext_crd_cd_lnd_ovdok.csv")
a = cd_ln.merge(ll,how = 'left',left_on = 'REPORT_ID',right_on = 'REPORT_ID')
cd_ln = a.merge(kk,how = 'left',left_on = 'REPORT_ID',right_on = 'REPORT_ID')
# print(cd_ln.shape[1]+cd_ln2.shape[1]+cd_ln3.shape[1]+cd_ln4.shape[1]+cd_ln5.shape[1]+cd_ln6.shape[1]+cd_ln7.shape[1])
# print(cd_ln.shape)
# cd_ln.head()  hd_reportok.csv
# cd_ln =  pd.read_csv("./contest_ext_crd_is_sharedebtok.csv")
print(cd_ln.shape)
cd_ln.head()

cd_ln (39970, 8)
ll (39970, 53)
kk (39970, 8)
(39970, 67)


,REPORT_ID,HOUSE_LOAN_COUNT,COMMERCIAL_LOAN_COUNT,OTHER_LOAN_COUNT,LOANCARD_COUNT,STANDARD_LOANCARD_COUNT,ANNOUNCE_COUNT,DISSENT_COUNT,rHOUSE_LOAN_COUNT_9.0,rHOUSE_LOAN_COUNT_10.0,...,rSTANDARD_LOANCARD_COUNT_9.0,rSTANDARD_LOANCARD_COUNT_10.0,rSTANDARD_LOANCARD_COUNT_29971.0,rHOUSE_LOAN_COUNT,rCOMMERCIAL_LOAN_COUNT,rOTHER_LOAN_COUNT,rFIRST_LOAN_OPEN_MONTH,rLOANCARD_COUNT,rFIRST_LOANCARD_OPEN_MONTH,rSTANDARD_LOANCARD_COUNT
0,8787,0.0,0.0,0.000000,0.017241,0.000000,0.0,0.0,1,0,...,1,0,0,0.822603,0.994754,0.115696,0.620087,0.156959,0.673574,0.850072
1,9410,0.0,0.0,0.013405,0.163793,0.000000,0.0,0.0,1,0,...,1,0,0,0.822603,0.994754,0.563894,0.260427,0.946734,0.718118,0.850072
2,25708,0.0,0.0,0.005362,0.051724,0.000000,0.0,0.0,1,0,...,1,0,0,0.822603,0.994754,0.341418,0.712813,0.507168,0.888222,0.850072
3,24820,0.0,0.0,0.000000,0.086207,0.000000,0.0,0.0,1,0,...,1,0,0,0.822603,0.994754,0.115696,0.620087,0.738346,0.511512,0.850072
4,22999,0.0,0.0,0.018767,0.120690,0.066667,0.0,0.0,1,0,...,0,1,0,0.822603,0.994754,0.655747,0.341675,0.870175,0.461995,0.953464


In [18]:
fulldata =  pd.read_csv("./basicOk.csv")
fulldata = fulldata.merge(cd_ln,how = 'left',left_on = 'REPORT_ID',right_on = 'REPORT_ID')
# fulldata.fillna(0,inplace = True)
print(fulldata.shape)
print(fulldata.isnull().sum().sort_values(ascending = False))


(40000, 130)
rSTANDARD_LOANCARD_COUNT          30
rOTHER_LOAN_COUNT_4.0             30
rFIRST_LOAN_OPEN_MONTH_7.0        30
rFIRST_LOAN_OPEN_MONTH_6.0        30
rFIRST_LOAN_OPEN_MONTH_5.0        30
rFIRST_LOAN_OPEN_MONTH_4.0        30
rFIRST_LOAN_OPEN_MONTH_3.0        30
rFIRST_LOAN_OPEN_MONTH_2.0        30
rFIRST_LOAN_OPEN_MONTH_1.0        30
rOTHER_LOAN_COUNT_29971.0         30
rOTHER_LOAN_COUNT_10.0            30
rOTHER_LOAN_COUNT_9.0             30
rOTHER_LOAN_COUNT_8.0             30
rOTHER_LOAN_COUNT_7.0             30
rOTHER_LOAN_COUNT_6.0             30
rOTHER_LOAN_COUNT_5.0             30
rOTHER_LOAN_COUNT_3.0             30
rFIRST_LOAN_OPEN_MONTH_9.0        30
rOTHER_LOAN_COUNT_2.0             30
rOTHER_LOAN_COUNT_1.0             30
rCOMMERCIAL_LOAN_COUNT_29971.0    30
rCOMMERCIAL_LOAN_COUNT_10.0       30
rHOUSE_LOAN_COUNT_29971.0         30
rHOUSE_LOAN_COUNT_10.0            30
rHOUSE_LOAN_COUNT_9.0             30
DISSENT_COUNT                     30
ANNOUNCE_COUNT           

In [56]:
fulldata = pd.read_csv("../merge/fulltable_RandomForstRegressor_fill.csv")
print(fulldata.shape)
# ll = pd.read_csv("../rank_discre_num/contest_ext_crd_is_creditcue_discre_ok.csv")
# print("ll",ll.shape)

# kk = pd.read_csv("../rank_discre_num/contest_ext_crd_is_creditcue_rank_ok.csv")
# print("kk",kk.shape)
# fulldata = fulldata.merge(ll,how = 'left',left_on = 'REPORT_ID',right_on = 'REPORT_ID')
# fulldata = fulldata.merge(kk,how = 'left',left_on = 'REPORT_ID',right_on = 'REPORT_ID')

(40000, 462)
ll (39970, 53)
kk (39970, 8)


In [49]:
def tianchong(i,label,data):
    rr = RandomForestRegressor(oob_score=True,random_state=3)
    mm = data.iloc[:,0:i]
#     print(mm.head())
    mm_notnull = mm.ix[mm[label].notnull()]
    mm_isnull = mm.ix[mm[label].isnull()]
    rr.fit(mm_notnull.iloc[:,0:-1],mm_notnull.iloc[:,-1])
    data[label].ix[data[label].isnull()] = rr.predict(mm_isnull.iloc[:,0:-1])
    return data

In [50]:
label = fulldata.columns
for i in range(1,len(label)):
    if fulldata[label[i]].isnull().any():
#         print("填充缺失值")
        fulldata = tianchong(i+1,label[i],fulldata)
#         print(fulldata.isnull().sum().sort_values(ascending = False))

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:725: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:725: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\llb\appdata\local\programs

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:725: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\llb\appdata\local\programs\py

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:725: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\llb\appdata\local\programs

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:725: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\llb\appdata\local\programs\py

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:725: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\llb\appdata\local\programs

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:725: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\llb\appdata\local\programs\py

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:725: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages

c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """
c:\users\llb\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:725: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\llb\appdata\local\programs

In [58]:
# print(fulldata.isnull().sum().sort_values(ascending = False))
# fulldata.to_csv("../merge/basic+cd_ln_RandomForestRegressorfill.csv",index = False)
# fulldata = pd.read_csv("../merge/fulltable_RandomForstRegressor_fill.csv")
# print(fulldata.shape)
# ll = pd.read_csv("../rank_discre_num/contest_ext_crd_is_creditcue_discre_ok.csv")
# print("ll",ll.shape)

# kk = pd.read_csv("../rank_discre_num/contest_ext_crd_is_creditcue_rank_ok.csv")
# print("kk",kk.shape)
# fulldata = fulldata.merge(ll,how = 'left',left_on = 'REPORT_ID',right_on = 'REPORT_ID')
# fulldata = fulldata.merge(kk,how = 'left',left_on = 'REPORT_ID',right_on = 'REPORT_ID')

# fulldata = pd.read_csv("../merge/fulldata_mean.csv")
print(fulldata.shape)
# fulldata.fillna(0,inplace = True)
fulldata.isnull().sum().sort_values(ascending = False)

(40000, 521)


rSTANDARD_LOANCARD_COUNT         0
guarantee_type_more_2.0_x        0
guarantee_type_more_7.0          0
guarantee_type_more_10.0         0
guarantee_type_more_15.0         0
guarantee_type_more_16.0         0
guarantee_type_more_17.0         0
guarantee_type_more_19.0         0
guarantee_type_more_22.0         0
guarantee_type_more_28.0         0
guarantee_type_less_2.0_x        0
guarantee_type_less_4.0          0
guarantee_type_less_7.0          0
guarantee_type_less_8.0          0
guarantee_type_less_9.0          0
guarantee_type_less_10.0         0
guarantee_type_less_15.0         0
guarantee_type_less_31.0         0
payment_rating_more_2.0          0
payment_rating_more_4.0          0
payment_rating_more_5.0          0
payment_rating_more_6.0          0
payment_rating_more_10.0         0
payment_rating_more_24.0         0
payment_rating_more_27.0         0
payment_rating_more_28.0         0
payment_rating_less_1.0          0
payment_rating_less_2.0          0
payment_rating_less_

In [32]:
fulldata.head()

,REPORT_ID,AGENT_0,AGENT_1,AGENT_2,AGENT_3,AGENT_4,AGENT_5,AGENT_6,AGENT_7,AGENT_8,...,LAST_MONTHS_max,LAST_MONTHS_min,LAST_MONTHS_mean,LAST_MONTHS_zongshu,LAST_MONTHS_num,AMOUNT_max,AMOUNT_min,AMOUNT_mean,AMOUNTt_zongshu,AMOUNT_num
0,8787,0,1,0,0,0,0,0,0,0,...,0.266667,0.0,0.15,0.019643,0.035802,0.040660,0.065798,0.050971,0.008108,0.037037
1,9410,0,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.008929,0.000000,0.001661,0.000295,0.001959,0.005405,0.000000
2,22999,0,1,0,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.008929,0.000000,0.001661,0.000295,0.001811,0.000000,0.000000
3,24820,0,1,0,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.008929,0.000000,0.001088,0.000960,0.001403,0.000000,0.000000
4,25708,1,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.00,0.017857,0.012346,0.001269,0.002420,0.002157,0.008108,0.012346


In [65]:
# print(fulldata.isnull().sum().sort_values(ascending = False))
basic = pd.read_csv("../input/contest_basic_train.tsv",sep = '\t')

print(basic.shape)
mm = pd.concat([basic['REPORT_ID'],basic['Y']],axis = 1)
train = mm.merge(fulldata,how = 'left',left_on = 'REPORT_ID',right_on = 'REPORT_ID')
# print(train.isnull().sum().sort_values(ascending = False))

print(train.columns)
train.head()

(30000, 11)
Index(['REPORT_ID', 'Y', 'AGENT_0', 'AGENT_1', 'AGENT_2', 'AGENT_3', 'AGENT_4',
       'AGENT_5', 'AGENT_6', 'AGENT_7', 'AGENT_8', 'AGENT_9', 'AGENT_10',
       'AGENT_11', 'AGENT_12', 'AGENT_13', 'AGENT_14', 'AGENT_15', 'AGENT_16',
       'AGENT_17', 'AGENT_18', 'AGENT_19', 'AGENT_20', 'IS_LOCAL_0',
       'IS_LOCAL_1', 'IS_LOCAL_2', 'EDU_LEVEL_0', 'EDU_LEVEL_1', 'EDU_LEVEL_2',
       'EDU_LEVEL_3', 'EDU_LEVEL_4', 'EDU_LEVEL_5', 'EDU_LEVEL_6',
       'EDU_LEVEL_7', 'EDU_LEVEL_8', 'EDU_LEVEL_9', 'MARRY_STATUS_0',
       'MARRY_STATUS_1', 'MARRY_STATUS_2', 'MARRY_STATUS_3', 'MARRY_STATUS_4',
       'MARRY_STATUS_5', 'MARRY_STATUS_6', 'HAS_FUND_0.0', 'HAS_FUND_1.0',
       'null_num', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'SALARY', 'day',
       'work_latitude', 'work_longitude', 'birth_latitude', 'birth_longitude',
       'is_diff1_0', 'is_diff1_1', 'is_diff1_0.1', 'is_diff1_1.1'],
      dtype='object')


,REPORT_ID,Y,AGENT_0,AGENT_1,AGENT_2,AGENT_3,AGENT_4,AGENT_5,AGENT_6,AGENT_7,...,SALARY,day,work_latitude,work_longitude,birth_latitude,birth_longitude,is_diff1_0,is_diff1_1,is_diff1_0.1,is_diff1_1.1
0,8787,0,0,1,0,0,0,0,0,0,...,0.333333,0.466667,0.894340,0.145498,0.551743,0.433991,0,1,1,0
1,9410,0,0,0,0,0,0,0,0,0,...,0.833333,0.400000,0.868396,0.230695,0.531979,0.567527,0,1,0,1
2,22999,0,0,1,0,0,0,0,0,0,...,0.500000,0.866667,0.900189,0.120483,0.647269,0.281032,0,1,0,1
3,24820,1,0,1,0,0,0,0,0,0,...,0.166667,0.766667,0.956415,0.311420,0.791381,0.783308,0,1,1,0
4,25708,0,1,0,0,0,0,0,0,0,...,0.166667,0.500000,0.900189,0.120483,0.558606,0.335660,1,0,0,1


In [66]:
trainx = train.iloc[:,2:]
trainy = train.iloc[:,1]
print(trainx.shape,trainy.shape)
trainx.head(),trainy.head()

(30000, 63) (30000,)


(   AGENT_0  AGENT_1  AGENT_2  AGENT_3  AGENT_4  AGENT_5  AGENT_6  AGENT_7  \
 0        0        1        0        0        0        0        0        0   
 1        0        0        0        0        0        0        0        0   
 2        0        1        0        0        0        0        0        0   
 3        0        1        0        0        0        0        0        0   
 4        1        0        0        0        0        0        0        0   
 
    AGENT_8  AGENT_9      ...         SALARY       day  work_latitude  \
 0        0        0      ...       0.333333  0.466667       0.894340   
 1        0        0      ...       0.833333  0.400000       0.868396   
 2        0        0      ...       0.500000  0.866667       0.900189   
 3        0        0      ...       0.166667  0.766667       0.956415   
 4        0        0      ...       0.166667  0.500000       0.900189   
 
    work_longitude  birth_latitude  birth_longitude  is_diff1_0  is_diff1_1  \
 0        0

In [26]:
feature = 0
for i in range(5):
    xgb2 = XGBClassifier(
            learning_rate =0.1,
            n_estimators=83,
            max_depth=5,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            objective= 'binary:logistic',
            nthread=4,
            scale_pos_weight=1,   #15:1
            StratifiedKFold = True,
            seed=27)
    xgb2.fit(trainx,trainy)
    feature += xgb2.feature_importances_
    print(i)
feature = feature/5


0
1
2
3
4


In [27]:
label = trainx.columns
# print(label)
print(label.shape)
print(feature)
label[feature>0].shape


(129,)
[0.0085213  0.01102757 0.         0.         0.00601504 0.
 0.00451128 0.         0.         0.         0.0075188  0.00401002
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.01102757 0.00250627
 0.00200501 0.00952381 0.02155389 0.         0.00802005 0.
 0.00651629 0.         0.         0.01102757 0.         0.
 0.         0.00451128 0.00501253 0.00601504 0.00200501 0.00501253
 0.00200501 0.01403509 0.00350877 0.00150376 0.00300752 0.00501253
 0.00350877 0.01553885 0.         0.         0.         0.02857143
 0.05363408 0.06716792 0.05714286 0.10325815 0.13032581 0.03157895
 0.00451128 0.01002506 0.         0.00300752 0.         0.04310777
 0.05413533 0.00401002 0.         0.         0.00200501 0.
 0.         0.         0.         0.         0.00050125 0.00501253
 0.00250627 0.00150376 0.00100251 0.00350877 0.00501253 0.00100251
 0.         0.         0.         0.00050125 0.00350877 0.00250627
 0.00250627 0.         0.006

(81,)

In [28]:
trainx = trainx[label[feature>0.0000000000001]]
print(trainx.shape)
trainx.head()

(30000, 81)


,AGENT_0,AGENT_1,AGENT_4,AGENT_6,AGENT_10,AGENT_11,IS_LOCAL_1,IS_LOCAL_2,EDU_LEVEL_0,EDU_LEVEL_1,...,rFIRST_LOANCARD_OPEN_MONTH_6.0,rFIRST_LOANCARD_OPEN_MONTH_7.0,rFIRST_LOANCARD_OPEN_MONTH_8.0,rFIRST_LOANCARD_OPEN_MONTH_9.0,rFIRST_LOANCARD_OPEN_MONTH_10.0,rSTANDARD_LOANCARD_COUNT_9.0,rOTHER_LOAN_COUNT,rFIRST_LOAN_OPEN_MONTH,rLOANCARD_COUNT,rFIRST_LOANCARD_OPEN_MONTH
0,0,1,0,0,0,0,1,0,0,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.115696,0.620087,0.156959,0.673574
1,0,0,0,0,0,1,1,0,1,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.563894,0.260427,0.946734,0.718118
2,0,1,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.655747,0.341675,0.870175,0.461995
3,0,1,0,0,0,0,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.115696,0.620087,0.738346,0.511512
4,1,0,0,0,0,0,1,0,0,1,...,0.0,0.0,0.0,1.0,0.0,1.0,0.341418,0.712813,0.507168,0.888222


In [337]:
trainy.value_counts()

0    28125
1     1875
Name: Y, dtype: int64

## Define a function for modeling and cross-validation
1.通过XGboost 自带的XGboostcv和earning 获得所需要的树的数目
2.其他关键参数通过 网格搜索 寻找
http://blog.csdn.net/q383700092/article/details/53763328

## Step 1- Find the number of estimators for a high learning rate

In [67]:
print("trainx.shape",trainx.shape)
print("trainy.shape",trainy.shape)
from sklearn.cross_validation import train_test_split
x_train,x_valid,y_train,y_valid = train_test_split(trainx,trainy,test_size = 0.2,random_state = 1)  
print("x_train.shape",x_train.shape)
print("x_valid.shape",x_valid.shape)
print("y_train.shape",y_train.shape)
print("y_valid.shape",y_valid.shape)

trainx.shape (30000, 63)
trainy.shape (30000,)
x_train.shape (24000, 63)
x_valid.shape (6000, 63)
y_train.shape (24000,)
y_valid.shape (6000,)


In [68]:
# x_train = trainx
# y_train = trainy
print(x_train.shape)
skf = StratifiedKFold(n_splits=5)
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        StratifiedKFold = True,
        seed=27)
#0.695099	0.008932	0.768925	0.003438       归一化后的效果  ，说明归一化还是有用 基础表最好的成绩（没加经纬度信息）
#0.871434	0.008775	0.928230	0.002010       上一个表（基础表） 加上经纬度与工作地与出生地是否在一个省

#0.870210	0.006927	0.952752	0.001073       cd_ln 未填充缺失值
#0.870119	0.007428	0.956189	0.000715       cd_ln 使用0填充缺失值
#0.868836  0.008394    0.943708    0.000619        cd_ln 使用随机森林填充缺失值

#0.870316	0.008369	0.930016	0.001322       加上主表   
#0.868805	0.008884	0.933789	0.002402       加上主表 后 删除没有特征贡献度维的结果  主表舍弃

#0.872930	0.009501	0.928759	0.002314       contest_ext_crd_is_creditcueok  缺失值使用0填充
#0.873482	0.008920	0.928747	0.002269       contest_ext_crd_is_creditcueok  缺失值不填
#0.872630	0.008383	0.936502	0.002553       contest_ext_crd_is_creditcueok   使用随机森林填充结果
#0.872620	0.009920	0.941726	0.002991       contest_ext_crd_is_creditcueok  缺失值不填,加上了离散化特征和排序特征



#0.866544	0.006012	0.938159	0.002178      contest_ext_crd_is_sharedebtok 未销户的结果（使用随机森林填充的结果）
#0.866851	0.006391	0.940869	0.001407      contest_ext_crd_is_sharedebtok 未销户的结果（未填充缺失值）
#0.869969	0.008164	0.944613	0.002065      contest_ext_crd_is_sharedebtok 只使用没有缺失值的部分
#0.869506	0.010020	0.952770	0.001094      contest_ext_crd_is_sharedebtok  删除了一些维度的结果。 

#0.873334	0.004927	0.984373	0.001025       全部表随机森林填充
# 0.871080	0.004091	0.979122	0.001468       全部表随机森林填充+contest_ext_crd_is_creditcueok  加上了离散化特征和排序特征(未填充缺失值)
#0.872549	0.004587	0.986786	0.000692       全部表随机森林填充+contest_ext_crd_is_creditcueok  加上了离散化特征和排序特征(随机森林缺失值)
#0.870467	0.004308	0.978745	0.001623       全部表随机森林填充+contest_ext_crd_is_creditcueok  加上了离散化特征和排序特征(随机森林缺失值)  
xgb_param = xgb1.get_xgb_params()
#构建稀疏矩阵，运行更快
xgtrain = xgb.DMatrix(x_train, label = y_train)
# xgtest =xgb.DMatrix(dtest[predictors].values)
mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=5,
    metrics='auc', early_stopping_rounds=50)
mm.iloc[-3:,:]

(24000, 63)


,test-auc-mean,test-auc-std,train-auc-mean,train-auc-std
122,0.866639,0.004426,0.928344,0.001247
123,0.866745,0.004414,0.928709,0.001239
124,0.866818,0.004567,0.929030,0.001208


In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
# from scikitplot import plotters as skplt

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC

cv = StratifiedKFold(n_splits=5, shuffle=True)

def estimate(estimator, name='estimator'):
    auc = cross_val_score(estimator, x_train, y_train, scoring='roc_auc', cv=cv).mean()
    accuracy = cross_val_score(estimator, x_train, y_train, scoring='accuracy', cv=cv).mean()
    recall = cross_val_score(estimator, x_train, y_train, scoring='recall', cv=cv).mean()

    print("{}: auc:{:f}, recall:{:f}, accuracy:{:f}".format(name, auc, recall, accuracy))


In [53]:
# estimate(XGBClassifier(
#         learning_rate =0.1,
#         n_estimators=mm.shape[0],
#         max_depth=5,
#         min_child_weight=6,
#         gamma=0.1,
#         subsample=0.8,
#         colsample_bytree=0.8,
#         reg_alpha=1e-05,
#         objective= 'binary:logistic',
#         nthread=4,
#         scale_pos_weight=1,
#         seed=27)
#          , 'XGBClassifier')     #0.8685
# estimate(RidgeClassifier(), 'RidgeClassifier')        #0.7969
# estimate(LogisticRegression(), 'LogisticRegression')   #0.797659
# estimate(RandomForestClassifier(), 'RandomForestClassifier')  #0.714345
# estimate(AdaBoostClassifier(), 'AdaBoostClassifier')  #0.838858
# estimate(SVC(), 'SVC')# too long to wait
# estimate(LinearSVC(), 'LinearSVC')


In [342]:
from sklearn.ensemble import VotingClassifier

estimators = []
estimators.append(('RidgeClassifier', RidgeClassifier()))
estimators.append(('LogisticRegression', LogisticRegression()))
estimators.append(('XGBClassifier', XGBClassifier(
        learning_rate =0.1,
        n_estimators=mm.shape[0],
        max_depth=5,
        min_child_weight=6,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=1e-05,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)))
# estimators.append(('AdaBoostClassifier', AdaBoostClassifier()))
# estimators.append(('RandomForestClassifier', RandomForestClassifier()))

# #voting: auc:0.794587, recall:0.000642, accuracy:0.944433

# voting = VotingClassifier(estimators = estimators, voting='soft')  #应该是比单独的xgb分类器要好很多
# estimate(voting, 'voting')

In [37]:
print(x_train.shape)   #0.690831	0.010714	0.776624	0.002014
print(y_train.shape)
# c = pd.Series(y_valid)
# print(c.value_counts())
# c.ix[c == 1] = 0
# print(c.value_counts())
# estimate(RidgeClassifier(), 'RidgeClassifier')        #0.7969
# estimate(LogisticRegression(), 'LogisticRegression')   #0.797659
# estimate(RandomForestClassifier(), 'RandomForestClassifier')  #0.714345
# estimate(AdaBoostClassifier(), 'AdaBoostClassifier')  #0.838858

xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=mm.shape[0],
        max_depth=5,
        min_child_weight=6,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=1e-05,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb2.fit(x_train,y_train)
pred1 = xgb2.predict_proba(x_valid)[:,1]
# pred1 = xgb2.predict_proba(x_valid)
auc = roc_auc_score(y_valid,pred1)
print("xgb auc",auc)

# ride = RidgeClassifier()
# ride.fit(x_train,y_train)
# # pred = xgb2.predict_proba(x_valid)[:,1]
# pred2 = ride.predict_proba(x_valid)[:,1]
# auc = roc_auc_score(y_valid,pred2)
# print("Ride auc",auc)

# ride = RidgeClassifier()
# ride.fit(x_train,y_train)
# # pred = xgb2.predict_proba(x_valid)[:,1]
# pred3 = ride.predict_proba(x_valid)[:,-1]
# auc = roc_auc_score(y_valid,pred3)
# print("ride auc",auc)

Log = LogisticRegression()
Log.fit(x_train,y_train)
# pred = xgb2.predict_proba(x_valid)[:,1]
pred4 = Log.predict_proba(x_valid)[:,-1]
auc = roc_auc_score(y_valid,pred4)
print("Log auc",auc)

# rr = RandomForestClassifier()
# rr.fit(x_train,y_train)
# # pred = xgb2.predict_proba(x_valid)[:,1]
# pred5 = rr.predict_proba(x_valid)[:,-1]
# auc = roc_auc_score(y_valid,pred5)
# print("rr auc",auc)
# print("total auc")

predttal = 0.9*pred1 + 0.1*pred4#+0.01*pred5
auc = roc_auc_score(y_valid,predttal)
print("total auc",auc)
# print()

(21000, 461)
(21000,)
xgb auc 0.8671208942459312


NameError: name 'LogisticRegression' is not defined

In [61]:
print(x_train.shape)   #0.690831	0.010714	0.776624	0.002014
print(y_train.shape)

#随机森林填充缺失值的结果（全部数据）
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=95,
        max_depth=5,
        min_child_weight=6,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=1e-05,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb1.fit(x_train,y_train)
pred1 = xgb1.predict_proba(x_valid)[:,1]
# pred1 = xgb2.predict_proba(x_valid)
auc = roc_auc_score(y_valid,pred1)
print("xgb1 auc",auc)

#均值填充缺失值的结果
xgb2 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=71,
        max_depth=5,
        min_child_weight=6,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=1e-05,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
xgb2.fit(x_train,y_train)
pred2 = xgb2.predict_proba(x_valid)[:,1]
# pred1 = xgb2.predict_proba(x_valid)
auc = roc_auc_score(y_valid,pred2)
print("xgb2 auc",auc)

# #0填充缺失值的结果
# xgb2 = XGBClassifier(
#         learning_rate =0.1,
#         n_estimators=95,
#         max_depth=5,
#         min_child_weight=6,
#         gamma=0.1,
#         subsample=0.8,
#         colsample_bytree=0.8,
#         reg_alpha=1e-05,
#         objective= 'binary:logistic',
#         nthread=4,
#         scale_pos_weight=1,
#         seed=27)
# xgb2.fit(x_train,y_train)
# pred1 = xgb2.predict_proba(x_valid)[:,1]
# # pred1 = xgb2.predict_proba(x_valid)
# auc = roc_auc_score(y_valid,pred1)
# print("xgb auc",auc)
predttal = 0.9*pred1 + 0.1*pred2#+0.01*pred5
auc = roc_auc_score(y_valid,predttal)
print("total auc",auc)

(21000, 526)
(21000,)
xgb1 auc 0.8695232948926959
xgb2 auc 0.8687586858914398
total auc 0.869609527420691


In [ ]:
# 这个是随机森林填充后降维到271维的最优数据
# xgb auc 0.8656555676564127
# Log auc 0.8075853722906698
# total auc 0.8658856602613786、

#随机森林填充后不降维的最优结果
xgb auc 0.8671208942459312
Log auc 0.7976350155009755
total auc 0.8662997016730364

# 下面通过网格搜索寻找其他参数

In [344]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols
# param_test1 = {
#     'max_depth':range(3,10,2),
#     'min_child_weight':range(1,6,2)
# }
# gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=mm.shape[0], max_depth=5,
#                                         min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
#                                         objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
#                        param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch1.fit(x_train,y_train)
# gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [345]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols

# print(" x_train", x_train.shape)
# print("y_train",y_train.shape)
# param_test2 = {
#     'max_depth':[4,5,6],
#     'min_child_weight':[4,5,6]
# }
# gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=mm.shape[0], max_depth=5,
#                                         min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
#                                         objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
#                        param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch2.fit(x_train,y_train)
# gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

In [346]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
# print(x_train.shape,y_train.shape)
# param_test2b = {
#     'min_child_weight':[6,8,10,12]
# }
# gsearch2b = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=mm.shape[0], max_depth=5,
#                                         min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
#                                         objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
#                        param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch2b.fit(x_train,y_train)
# gsearch2b.grid_scores_, gsearch2b.best_params_, gsearch2b.best_score_

In [347]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols
# param_test3 = {
#     'gamma':[i/10.0 for i in range(0,5)]
# }
# gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=mm.shape[0], max_depth=5,
#                                         min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
#                                         objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
#                        param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch3.fit(x_train,y_train)
# gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [348]:
# skf = StratifiedKFold(n_splits=5)
# xgb1 = XGBClassifier(
#         learning_rate =0.1,
#         n_estimators=1000,
#         max_depth=5,
#         min_child_weight=6,
#         gamma=0.1,
#         subsample=0.8,
#         colsample_bytree=0.8,
#         objective= 'binary:logistic',
#         nthread=4,
#         scale_pos_weight=1,
#         StratifiedKFold = True,
#         seed=27)


# xgb_param = xgb1.get_xgb_params()
# #构建稀疏矩阵，运行更快
# xgtrain = xgb.DMatrix(x_train, label = y_train)
# # xgtest =xgb.DMatrix(dtest[predictors].values)
# mm = xgb.cv(xgb_param, xgtrain, num_boost_round=1000, nfold=5,
#     metrics='auc', early_stopping_rounds=50)
# # mm

Tune subsample and colsample_bytree

In [349]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols
# param_test4 = {
#     'subsample':[i/10.0 for i in range(6,10)],
#     'colsample_bytree':[i/10.0 for i in range(6,10)]
# }
# gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=mm.shape[0], max_depth=5,
#                                         min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
#                                         objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
#                        param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch4.fit(x_train,y_train)
# gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

tune subsample:

In [350]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
# param_test5 = {
#     'subsample':[i/100.0 for i in range(75,90,5)],
#     'colsample_bytree':[i/100.0 for i in range(75,90,5)]
# }
# gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=mm.shape[0], max_depth=5,
#                                         min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
#                                         objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
#                        param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch5.fit(x_train,y_train)
# gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

Got the same value as assument and no change requried.

Try regularization:  (防止过拟合的)

In [351]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols
# param_test6 = {
#     'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
# }
# gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=mm.shape[0], max_depth=5,
#                                         min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
#                                         objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
#                        param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch6.fit(x_train,y_train)
# gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

In [352]:
# #Grid seach on subsample and max_features
# #Choose all predictors except target & IDcols
# param_test7 = {
#     'reg_alpha':[0, 0.00001, 0.00005, 0.01, 0.05]
# }
# gsearch7 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=mm.shape[0], max_depth=5,
#                                         min_child_weight=6, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
#                                         objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
#                        param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch7.fit(x_train,y_train)
# gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_

In [353]:
# xgb3 = XGBClassifier(
#         learning_rate =0.1,
#         n_estimators=54,
#         max_depth=5,
#         min_child_weight=6,
#         gamma=0.1,
#         subsample=0.8,
#         colsample_bytree=0.8,
#         reg_alpha=1e-05,
#         objective= 'binary:logistic',
#         nthread=4,
#         scale_pos_weight=1,
#         seed=27)
# param_test7 = {
#     'n_estimators':range(mm.shape[0]-10,mm.shape[0]+10,1)
# }
# gsearch7 = GridSearchCV(estimator = xgb3,
#                        param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch7.fit(x_train,y_train)
# gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_